In [5]:
%matplotlib inline
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader, TensorDataset, Subset, Dataset
from sklearn.model_selection import train_test_split, KFold
from PIL import Image
import shutil
import pandas
from torchvision.io import read_image

cudnn.benchmark = True
plt.ion()   # interactive mode
#https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [26]:
def test_model(model, dataset, labels):
    since = time.time()

    # Each epoch has a training and validation phase
    #model.eval()

    running_corrects = 0

    # Iterate over data.
    for idx in range(len(labels)):
        
        image = Image.fromarray(dataset[:, :, idx], mode = 'L').resize((320, 320))
        label = labels[idx]
        if not isinstance(label, list):
            label = [label]
        # zero the parameter gradients
        #optimizer.zero_grad()

        # forward
        output = model(image, size=320)
        predictions = output.pandas().xyxy[0].sort_values('xmin').loc[:, "class"].tolist()
        predictions_char_arr = [str(prediction) if prediction != 10 else 'u' for prediction in predictions]
        predictions_str = "".join(predictions_char_arr)
        print("Number of predictions: ", len(predictions_str), "Prediction: ", predictions_str)
        running_corrects += (predictions == label)  
        #Calculate accuracy for each label; do average (varies by size of label): microaverage (use if imbalance)
        #Calculate accuracy for each specific prediction: macroaverage (try this)
        #output.print()
        #_, preds = torch.max(outputs, 1)
        #print(preds)
        #loss = criterion(output, label)

        # statistics
        #running_loss += loss.item() * inputs.size(0)
        #running_corrects += torch.sum(preds == labels.data)

    #loss = running_loss / dataset_test_size
    acc = running_corrects / len(labels)

    print(f'Acc: {acc:.4f}')

    time_elapsed = time.time() - since
    print(f'Testing complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    return model

In [27]:
preceding_path = ""
dataset = np.load(os.path.join(preceding_path, 'data_train.npy')).reshape((300, 300, 9032))
labels = np.load(os.path.join(preceding_path, 't_train_corrected.npy'))
model_ft = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\justin.rossiter/.cache\torch\hub\master.zip
requirements: YOLOv5 requirements "gitpython" "tqdm>=4.64.0" not found, attempting AutoUpdate...
requirements:  Command 'pip install "gitpython" "tqdm>=4.64.0"  ' returned non-zero exit status 1.
YOLOv5  2022-11-30 Python-3.8.8 torch-1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)

Fusing layers... 
Model summary: 157 layers, 7039792 parameters, 0 gradients
Adding AutoShape... 


In [28]:
test_model(model_ft, dataset, labels)

Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  3
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  5
Number of predic

Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  3
Number of predictions:  1 Prediction:  3
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  3
Number of predictions:  1 Prediction:  3
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  8
Number of predic

Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  6
Number of predic

Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  3
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  5
Number of predictions:  2 Prediction:  u0
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  4
Number of predi

Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  3
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  3
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  2
Number of predic

...

Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  3
Number of predictions:  1 Prediction:  3
Number of predictions:  2 Prediction:  87
Number of predictions:  1 Prediction:  3
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  3
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  1
Number of predi

Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  3
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  0
Number of predic

Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  3
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  3
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  1
Number of predictions:  1 Prediction:  4
Number of predic

Number of predictions:  1 Prediction:  0
Number of predictions:  1 Prediction:  2
Number of predictions:  2 Prediction:  78
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  8
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  5
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  9
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  6
Number of predictions:  1 Prediction:  4
Number of predictions:  1 Prediction:  2
Number of predictions:  1 Prediction:  7
Number of predictions:  1 Prediction:  9
Number of predi

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  